In [1]:
import pandas as pd
import numpy as np
from sklearn import model_selection
from sklearn.ensemble import RandomForestRegressor
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler, Normalizer, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error

df = pd.read_excel('C:/Anaconda3/projects/oled/oled.xlsx', sheetname='pholed_model')
df = df[list(df.columns)[1:-1] + ['cd/A']]
df.dropna(axis=0, inplace=True)

X = df[list(df.columns)[:-1]]
y = df[['cd/A']]

X.head()

,homo_HIL,thickness_HIL,lumo_HTL1,homo_HTL1,thickness_HTL1,lumo_HTL2,homo_HTL2,thickness_HTL2,lumo_HTL3,homo_HTL3,...,triplet_ETL,total_ETL_thickness,lumo_ETL1,homo_ETL1,thickness_ETL1,lumo_ETL2,homo_ETL2,thickness_ETL2,cathode_workfunction,thickness_EIL
0,5.2,25.0,3.3,5.2,0,3.3,5.2,0,3.3,5.2,...,2.6,35,2.8,6.2,0,2.8,6.2,35.0,2.8,10.0
1,5.2,25.0,3.3,5.2,0,3.3,5.2,0,3.3,5.2,...,2.6,35,2.8,6.2,0,2.8,6.2,35.0,2.8,10.0
2,5.2,25.0,3.3,5.2,0,3.3,5.2,0,3.3,5.2,...,2.6,35,2.8,6.2,0,2.8,6.2,35.0,2.8,10.0
3,5.2,25.0,3.3,5.2,0,3.3,5.2,0,3.3,5.2,...,2.6,35,2.8,6.2,0,2.8,6.2,35.0,2.8,10.0
4,5.2,25.0,3.3,5.2,0,3.3,5.2,0,3.3,5.2,...,2.6,35,2.8,6.2,0,2.8,6.2,35.0,2.8,10.0


In [2]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.25, random_state=12)

# filename = 'adaboost_model.sav'
filename = 'pholed_rf_model.sav'

# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
print('R-squared: ', loaded_model.score(X_test, y_test))

R-squared:  0.87869391923


In [82]:
# new_df = pd.read_excel('C:/Anaconda3/projects/oled/generated_devices.xlsx', sheetname='Sheet1')
new_df = pd.read_pickle('C:/Anaconda3/projects/oled/devices.pkl')

new_X = new_df[:300000]
# print(new_df.shape)

y_pred = loaded_model.predict(new_X)
y_pred = [round(float(i), 2) for i in y_pred]

new_X['cd/a'] = y_pred
device = new_X.loc[(new_X['cd/a'] >= 35) & (new_X['cd/a'] < 40)]
# device = new_X.loc[new_X['cd/a'] < 10]
device.shape

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(93269, 32)

In [83]:
# new_df = pd.read_excel('C:/Anaconda3/projects/oled/generated_devices.xlsx', sheetname='Sheet1')
new_df = pd.read_pickle('C:/Anaconda3/projects/oled/devices.pkl')

new_X = new_df[300000:600000]
# print(new_df.shape)

y_pred = loaded_model.predict(new_X)
y_pred = [round(float(i), 2) for i in y_pred]

new_X['cd/a'] = y_pred
device_b = new_X.loc[(new_X['cd/a'] >= 35) & (new_X['cd/a'] < 40)]
device = device.append(device_b)
device.shape

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(123958, 32)

In [84]:
# new_df = pd.read_excel('C:/Anaconda3/projects/oled/generated_devices.xlsx', sheetname='Sheet1')
new_df = pd.read_pickle('C:/Anaconda3/projects/oled/devices.pkl')

new_X = new_df[600000:900000]
# print(new_df.shape)

y_pred = loaded_model.predict(new_X)
y_pred = [round(float(i), 2) for i in y_pred]

new_X['cd/a'] = y_pred
device_b = new_X.loc[(new_X['cd/a'] >= 35) & (new_X['cd/a'] < 40)]
device = device.append(device_b)
device.shape

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(151828, 32)

In [85]:
# new_df = pd.read_excel('C:/Anaconda3/projects/oled/generated_devices.xlsx', sheetname='Sheet1')
new_df = pd.read_pickle('C:/Anaconda3/projects/oled/devices.pkl')

new_X = new_df[900000:]
# print(new_df.shape)

y_pred = loaded_model.predict(new_X)
y_pred = [round(float(i), 2) for i in y_pred]

new_X['cd/a'] = y_pred
device_b = new_X.loc[(new_X['cd/a'] >= 35) & (new_X['cd/a'] < 40)]
device = device.append(device_b)
device.shape

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(178696, 32)

In [86]:
device.columns = ['homo_HIL','thick_HIL','lumo_HTL1','homo_HTL1','thickness_HTL1','lumo_HTL2','homo_HTL2','thickness_HTL2',
                  'lumo_HTL3','homo_HTL3','thickness_HTL3','total_HTL_thickness','triplet_HTL','LUMO_EML','HOMO_EML',
                  'triplet_HOST','thickness_EML','lumo_dopant','homo_dopant','triplet_dopant','dopant_percentage','triplet_ETL',
                  'total ETL','lumo_ETL1','homo_ETL1','thickness_ETL1','lumo_ETL2','homo_ETL2','thickness_ETL2','wf_1',
                  'thickness_EIL','cd/a']
device.head()

,homo_HIL,thick_HIL,lumo_HTL1,homo_HTL1,thickness_HTL1,lumo_HTL2,homo_HTL2,thickness_HTL2,lumo_HTL3,homo_HTL3,...,total ETL,lumo_ETL1,homo_ETL1,thickness_ETL1,lumo_ETL2,homo_ETL2,thickness_ETL2,wf_1,thickness_EIL,cd/a
1040,5.2,60,2.4,6.1,40,2.4,6.1,0,2.4,6.1,...,35,2.73,6.12,0,2.73,6.12,35,2.9,10,38.49
1041,5.2,60,2.4,6.1,40,2.4,6.1,0,2.4,6.1,...,35,2.73,6.12,0,2.73,6.12,35,2.2,1,38.03
1042,5.2,60,2.4,6.1,40,2.4,6.1,0,2.4,6.1,...,35,2.73,6.12,0,2.73,6.12,35,3.5,1,37.46
1043,5.2,60,2.4,6.1,40,2.4,6.1,0,2.4,6.1,...,35,2.73,6.12,0,2.73,6.12,35,2.8,1,38.03
1045,5.2,60,2.4,6.1,40,2.4,6.1,0,2.4,6.1,...,35,3.32,6.67,0,3.32,6.67,35,2.9,10,36.10


In [ ]:
# from openpyxl import load_workbook

# with pd.ExcelWriter('C:\Anaconda3\projects\oled\device_40.xlsx', engine='openpyxl') as writer:
#     writer.book = load_workbook('C:\Anaconda3\projects\oled\device_50.xlsx')
#     device.to_excel(writer,  sheet_name='35-40', index=True)
# writer.save()

In [87]:
device.to_pickle('C:/Anaconda3/projects/oled/35-40.pkl')

In [88]:
df = pd.read_pickle('C:/Anaconda3/projects/oled/35-40.pkl')
df.describe()

,homo_HIL,thick_HIL,lumo_HTL1,homo_HTL1,thickness_HTL1,lumo_HTL2,homo_HTL2,thickness_HTL2,lumo_HTL3,homo_HTL3,...,total ETL,lumo_ETL1,homo_ETL1,thickness_ETL1,lumo_ETL2,homo_ETL2,thickness_ETL2,wf_1,thickness_EIL,cd/a
count,178696.000000,178696.000000,178696.000000,178696.000000,178696.0,178696.000000,178696.000000,178696.0,178696.000000,178696.000000,...,178696.0,178696.000000,178696.000000,178696.0,178696.000000,178696.000000,178696.0,178696.000000,178696.000000,178696.000000
mean,6.300866,33.492697,2.301666,5.750687,40.0,2.301666,5.750687,0.0,2.301666,5.750687,...,35.0,2.676480,6.508365,0.0,2.676480,6.508365,35.0,2.859601,3.263738,36.743446
std,1.787347,26.225848,0.160751,0.286194,0.0,0.160751,0.286194,0.0,0.160751,0.286194,...,0.0,0.249328,0.323348,0.0,0.249328,0.323348,0.0,0.500772,3.938622,1.312454
min,5.200000,1.000000,1.900000,5.100000,40.0,1.900000,5.100000,0.0,1.900000,5.100000,...,35.0,2.260000,5.850000,0.0,2.260000,5.850000,35.0,2.200000,0.000000,35.000000
25%,5.200000,5.000000,2.200000,5.700000,40.0,2.200000,5.700000,0.0,2.200000,5.700000,...,35.0,2.500000,6.300000,0.0,2.500000,6.300000,35.0,2.200000,1.000000,35.640000
50%,5.200000,40.000000,2.400000,5.800000,40.0,2.400000,5.800000,0.0,2.400000,5.800000,...,35.0,2.700000,6.560000,0.0,2.700000,6.560000,35.0,2.800000,1.000000,36.440000
75%,6.100000,60.000000,2.400000,5.900000,40.0,2.400000,5.900000,0.0,2.400000,5.900000,...,35.0,2.730000,6.800000,0.0,2.730000,6.800000,35.0,2.900000,10.000000,37.660000
max,9.700000,60.000000,2.500000,6.100000,40.0,2.500000,6.100000,0.0,2.500000,6.100000,...,35.0,3.320000,7.000000,0.0,3.320000,7.000000,35.0,4.280000,10.000000,39.990000


In [20]:
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

new_y = new_df['cd/A']

r2 = r2_score(new_y, y_pred)
mse = mean_squared_error(new_y, y_pred)

print(r2, mse, np.sqrt(mse))

-3.27889023847 34.395431293 5.86476182747


In [14]:
print("Instance 0 prediction:", loaded_model.predict(new_X[0]))
print("Instance 1 prediction:", loaded_model.predict(new_X[1]))

Instance 0 prediction: [ 19.587]
Instance 1 prediction: [ 23.39179487]


C:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


### Tree interpreter
Using tree interpreter, instruction available on this blog post http://blog.datadive.net/random-forest-interpretation-with-scikit-learn/

In [16]:
from treeinterpreter import treeinterpreter as ti

prediction, bias, contributions = ti.predict(loaded_model, new_X[:2])
names = list(X)
for i in range(len(new_X[:2])):
    print("Instance", i)
    print("Bias (trainset mean)", bias[i])
    print("Feature contributions:")
    for c, feature in sorted(zip(contributions[i], 
                                 names), 
                             key=lambda x: -abs(x[0])):
        print(feature, round(c, 2))
    print("-"*20 )

Instance 0
Bias (trainset mean) 26.7501777778
Feature contributions:
thickness_EIL 18.21
triplet_ETL 2.32
homo_HIL 2.02
thickness_HTL1 0.75
thickness_ETL2 0.74
homo_dopant 0.72
triplet_HOST -0.67
LUMO_EML 0.67
lumo_HTL1 -0.36
total_ETL_thickness 0.28
homo_ETL1 -0.22
total_HTL_thickness 0.17
homo_HTL3 0.17
homo_HTL1 -0.14
thickness_EML 0.1
dopant_percentage 0.08
homo_ETL2 -0.04
homo_HTL2 -0.01
lumo_HTL3 -0.0
thickness_HIL 0.0
lumo_HTL2 0.0
thickness_HTL2 0.0
thickness_HTL3 0.0
triplet_HTL 0.0
HOMO_EML 0.0
lumo_dopant 0.0
triplet_dopant 0.0
lumo_ETL1 0.0
thickness_ETL1 0.0
lumo_ETL2 0.0
cathode_workfunction 0.0
--------------------
Instance 1
Bias (trainset mean) 26.7501777778
Feature contributions:
thickness_EIL 18.21
triplet_ETL 2.32
homo_HIL 2.02
thickness_HTL1 0.75
thickness_ETL2 0.74
homo_dopant 0.72
triplet_HOST -0.67
LUMO_EML 0.67
lumo_HTL1 -0.36
total_ETL_thickness 0.28
homo_ETL1 -0.22
total_HTL_thickness 0.17
homo_HTL3 0.17
homo_HTL1 -0.14
thickness_EML 0.1
dopant_percentage 0.0

In [26]:
import numpy as np

print(prediction)
print(bias + np.sum(contributions, axis=1))

[ 19.587       23.39179487  21.45        20.76333333  23.39179487
  26.42179487]
[ 19.587       23.39179487  21.45        20.76333333  23.39179487
  26.42179487]


In [27]:
# try using scikit decision path
loaded_model.decision_path(new_X)

(<6x1310 sparse matrix of type '<class 'numpy.int64'>'
 	with 461 stored elements in Compressed Sparse Row format>,
 array([   0,  121,  250,  387,  528,  663,  794,  921, 1058, 1191, 1310], dtype=int32))